In [1]:
import warnings
import pickle

import numpy as np
import pandas as pd

from scipy.signal import butter, filtfilt, freqz
from scipy.special import expit
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.nonparametric.smoothers_lowess import lowess

import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
sales = pd.read_csv("../data/preprocessed/sales.csv")
sell_prices = pd.read_csv("../data/preprocessed/sell_prices.csv")

In [ ]:
from scipy.signal import butter, filtfilt, freqz

def estimate_periods_with_fourier(sale, start_col, save_plot=False):
    # start_col 이후의 데이터만 선택
    cols = [col for col in sale.index if col.startswith('d_') and int(col.split('_')[1]) >= int(start_col.split('_')[1])]
    sale_values = sale[cols].values.astype(float)

    # High-pass 필터 파라미터 계산 및 필터 적용
    cutoff, order = 0.01, 10
    nyquist = 0.5
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    filtered_sale_values = filtfilt(b, a, sale_values)  # 직접 필터 적용

    # 필터링된 신호의 FFT
    fft = np.fft.fft(filtered_sale_values)
    frequencies = np.fft.fftfreq(len(filtered_sale_values), d=1)
    magnitudes = np.abs(fft)

    # 양의 주파수만 선택하고 진폭순 정렬
    positive_mask = frequencies > 0
    frequencies_pos = frequencies[positive_mask]  
    magnitudes_pos = magnitudes[positive_mask]
    sorted_indices = np.argsort(magnitudes_pos)[::-1]
    
    # 주파수 정렬
    sorted_frequencies = frequencies_pos[sorted_indices]
    sorted_magnitudes = magnitudes_pos[sorted_indices]

    # 주기 계산
    raw_periods = [1/f for f in sorted_frequencies if 1/f < len(sale_values)//2]
    
    # 유사한 주기 병합
    merged_periods = []
    
    for period in raw_periods:
        # 유사한 주기가 이미 있는지 확인
        if not any(abs(period - p) <= max(1, p * 0.05) for p in merged_periods):
            merged_periods.append(period)

    # 주기별 누적 SNR 계산
    snr_progression = []
    reconstructed_values_list = []
    
    for i in range(1, len(merged_periods) + 1):
        # i개의 주기만 사용하여 신호 복원
        current_periods = merged_periods[:i]
        reconstruct_fft = np.zeros_like(fft, dtype=complex)
        
        for period in current_periods:
            freq = 1/period
            freq_idx = np.argmin(np.abs(frequencies - freq))
            
            # 원본 진폭과 위상 보존
            reconstruct_fft[freq_idx] = fft[freq_idx]
            reconstruct_fft[-freq_idx] = fft[-freq_idx]
        
        # 복원 신호 생성
        reconstructed_values = np.fft.ifft(reconstruct_fft).real
        reconstructed_values_list.append(reconstructed_values)
        
        # SNR 계산
        original_energy = np.sum(np.abs(filtered_sale_values) ** 2)
        error_energy = np.sum(np.abs(filtered_sale_values - reconstructed_values) ** 2)
        current_snr = 10 * np.log10(original_energy / error_energy)
        snr_progression.append(current_snr)

    if save_plot:
        plt.figure(figsize=(15, 15))
        
        # SNR 진행도 플롯
        plt.subplot(211)
        plt.plot(range(1, len(snr_progression) + 1), snr_progression)
        plt.xlabel('Number of Periods Used')
        plt.ylabel('SNR (dB)')
        plt.title(f'SNR Progression: {sale["state_id"]}, {sale["item_id"]}')
        
        # 원본 신호와 복원 신호 비교 플롯
        plt.subplot(212)
        plt.plot(filtered_sale_values, label='Original Signal', alpha=0.7)
        plt.plot(reconstructed_values_list[-1], label='Reconstructed Signal', alpha=0.7)

        plt.xlabel('Time')
        plt.ylabel('Value')
        plt.title('Signal Reconstruction Comparison')
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(f"../data/fourier/snr_progression/{sale['state_id']}_{sale['item_id']}.png")
        plt.close()

    return merged_periods, snr_progression


def analyze_period_with_fourier(sales, first_sales_column_dict, save_result=False, save_plot=False):
    # Fundamental Period, Period Strength
    fourier_results = {}

    for idx in sales.index:
        sale = sales.iloc[idx]
        key = (sale['state_id'], sale['item_id'])

        # 시작 컬럼 가져오기
        start_col = first_sales_column_dict.get(key)

        # Fourier 분석 수행
        periods, snr_progression = estimate_periods_with_fourier(sale, start_col, save_plot=save_plot)

        # 결과 저장
        fourier_results[key] = {
            'periods': periods,
            'snr_progression': snr_progression,
        }

    # 결과 저장 (Pickle 파일)
    if save_result:
        with open('../data/fourier/results.pkl', 'wb') as f:
            pickle.dump(fourier_results, f)

    return fourier_results

In [3]:
# sell_prices의 각 행에서 NaN이 끝나고 처음으로 숫자인 column을 찾기
def create_first_sales_column_dict(df, save_result=False):
    first_sales_column_dict = {}
    
    for idx in df.index:
        row = df.iloc[idx]
        # state_id와 item_id로 키 튜플 생성
        key = (row['state_id'], row['item_id'])
        
        # NaN 값들의 위치를 찾습니다
        nan_mask = row.isna()
        
        if not nan_mask.any():
            # NaN이 없는 경우 첫 번째 데이터 컬럼('d_1')을 저장
            first_sales_column_dict[key] = 'd_1'
        else:
            # 마지막 NaN의 위치를 찾습니다
            last_nan_idx = nan_mask[::-1].idxmax()
            # 마지막 NaN의 위치 이후의 첫 번째 숫자가 있는 컬럼을 찾습니다
            last_nan_position = row.index.get_loc(last_nan_idx)
            first_number_col = row.index[last_nan_position + 1]
            first_sales_column_dict[key] = first_number_col
    
    if save_result:
        with open('../data/preprocessed/first_sales_column_dict.pkl', 'wb') as f:
            pickle.dump(first_sales_column_dict, f)
            
    return first_sales_column_dict

In [4]:
# 저장
# first_sales_column_dict = create_first_sales_column_dict(sell_prices, save_result==True)

# 로드
with open('../data/preprocessed/first_sales_column_dict.pkl', 'rb') as f:
    first_sales_column_dict = pickle.load(f)

### fourier transform

In [ ]:
def estimate_periods_with_fourier(sale, start_col, save_plot=False):
    # ...existing code...
    
    # 주기 계산 및 중복 제거
    raw_periods = [1/f for f in sorted_frequencies if 1/f < len(sale_values)//2]
    
    # 유사한 주기 병합 - 수정된 버전
    merged_periods = []
    merged_magnitudes = []
    
    if raw_periods:
        # 주기와 진폭을 함께 정렬
        period_magnitude_pairs = list(zip(raw_periods, sorted_magnitudes))
        period_magnitude_pairs.sort(key=lambda x: x[1], reverse=True)  # 진폭 기준 내림차순 정렬
        
        for period, magnitude in period_magnitude_pairs:
            period = int(round(period))
            # 이미 존재하는 유사한 주기가 있는지 확인
            similar_exists = any(abs(period - p) <= max(1, p * 0.05) for p in merged_periods)
            
            if not similar_exists:
                merged_periods.append(period)
                merged_magnitudes.append(magnitude)
    
    # 최종 결과 정렬
    merged_periods = sorted(list(set(merged_periods)))
    
    # ...existing code...


In [8]:
# 저장
fourier_results = analyze_period_with_fourier(sales, first_sales_column_dict, save_result=True, save_plot=False)

# 로드
with open('../data/fourier/results.pkl', 'rb') as f:
    fourier_results = pickle.load(f)

### log differencing

In [ ]:
def calculate_sell_price_changes_with_log_differencing(sell_prices, first_sales_column_dict, save_result=False, save_plot=False):

    # 데이터 복사
    log_differenced_sell_prices = sell_prices.copy()

    # 행별로 시작 컬럼부터 차분 수행
    for idx, sell_price in sell_prices.iterrows():
        # 시작 컬럼 가져오기
        start_col = first_sales_column_dict.get(tuple(sell_price[:2]))
        
        # 시작 컬럼 이후 데이터 선택
        start_index = sell_prices.columns.get_loc(start_col)
        sell_price_values = np.array(sell_price.iloc[start_index:].values, dtype=np.float64)

        # 로그 변환 및 차분 계산
        logged_sell_price_values = np.log(sell_price_values + 1e-9)  # 로그 계산 시 0 방지
        log_differenced_sell_price_values = np.diff(logged_sell_price_values, prepend=logged_sell_price_values[0])

        # 결과 저장
        log_differenced_sell_prices.iloc[idx, start_index:] = log_differenced_sell_price_values

        # 시각화
        if save_plot:
            fig, axs = plt.subplots(2, 1, figsize=(15, 10))

            axs[0].plot(sell_price_values, label=f"Original Sell Prices: {sell_price['state_id']}, {sell_price['item_id']}")
            axs[0].set_xlabel("Time")
            axs[0].set_ylabel("Sell Prices")
            axs[0].legend()

            axs[1].plot(log_differenced_sell_price_values, label=f"Log-Differenced Sell Prices: {sell_price['state_id']}, {sell_price['item_id']}")
            axs[1].set_xlabel("Time")
            axs[1].set_ylabel("Log-Differenced Sell Prices")
            axs[1].legend()

            plt.tight_layout()
            plt.savefig(f"../data/log_differencing/plot/{sell_price['state_id']}_{sell_price['item_id']}.png")
            plt.close()

    if save_result:
        log_differenced_sell_prices.to_csv("../data/log_differencing/log_differenced_sell_prices.csv", index=False)

    return log_differenced_sell_prices

In [ ]:
# 저장
# log_differenced_sell_prices = calculate_sell_price_changes_with_log_differencing(sell_prices, first_sales_column_dict, save_result=True, save_plot=True)

# 로드
log_differenced_sell_prices = pd.read_csv("../data/log_differencing/log_differenced_sell_prices.csv")

In [ ]:
def analyze_stl_decomposition(data, period):
    """STL 분해 수행 및 분석"""
    from statsmodels.tsa.seasonal import STL
    
    stl = STL(data, period=period)
    result = stl.fit()
    
    return {
        'trend': result.trend,
        'seasonal': result.seasonal,
        'resid': result.resid,
        'strength_of_seasonality': 1 - result.resid.var() / (result.seasonal + result.resid).var()
    }

def optimize_period_selection(data, periods, snr_threshold=10):
    """SNR 기반 최적 주기 선택"""
    selected_periods = []
    current_snr = 0
    
    for period in sorted(periods):
        # 해당 주기로 신호 재구성
        reconstructed = reconstruct_signal_with_period(data, period)
        new_snr = calculate_snr(data, reconstructed)
        
        if new_snr - current_snr > snr_threshold:
            selected_periods.append(period)
            current_snr = new_snr
            
    return selected_periods

def analyze_residual_periodicity(residuals):
    """잔차의 주기성 분석"""
    fft = np.fft.fft(residuals)
    frequencies = np.fft.fftfreq(len(residuals))
    
    # 주요 주파수 성분 식별
    magnitude_threshold = np.percentile(np.abs(fft), 95)
    significant_freqs = frequencies[np.abs(fft) > magnitude_threshold]
    
    return [1/freq for freq in significant_freqs if freq != 0]
